## VWAP Mean-Reversion Strategy

### Evan Schreiner - Feb. 2026

I am going to be implementing this strategy exactly as is outlined in Max Meystrick's VWAP paper on 02/02/2026 found in the TigerQuant Drive.

To start, I will start by fetching minute price data for AMD from Yahoo Finance.

*Note: Yahoo Finance only lets you download minute pricing data for stocks within the past 30 days, and only 8 days at a time.*

In [9]:
import numpy as np
import pandas
import yfinance as yf
import datetime as dt

In [10]:
###
# Alpaca Data Fetch
# Uncomment if your prefer this
###

# import os
# from dotenv import load_dotenv
# from alpaca.data.historical import StockHistoricalDataClient
# from alpaca.data.requests import StockBarsRequest
# from alpaca.data.timeframe import TimeFrame

# load_dotenv()
# API_KEY = os.getenv('API_KEY')
# SECRET_KEY = os.getenv('SECRET_KEY')

# data_client = StockHistoricalDataClient(API_KEY, SECRET_KEY)

# request_params = StockBarsRequest(
#     symbol_or_symbols=['AMD'],
#     timeframe=TimeFrame.Minute,
#     start='2025-01-01'
# )

# bars = data_client.get_stock_bars(request_params)
# data = bars.df
# data

In [11]:
# Fetch Data
amd_data = yf.download('AMD', start='2026-01-24', end='2026-01-31', interval='1m', ignore_tz=True, multi_level_index=False, )
amd_data.sort_index(ascending=True, inplace=True)
amd_data

[*********************100%***********************]  1 of 1 completed


,Close,High,Low,Open,Volume
Datetime,,,,,
2026-01-26 09:30:00,256.459991,258.289886,255.450104,256.744995,1086751
2026-01-26 09:31:00,254.625000,256.760010,254.369995,256.320007,178767
2026-01-26 09:32:00,255.300003,255.869995,254.399994,254.625000,156578
2026-01-26 09:33:00,253.750107,255.513397,253.380005,255.490005,160508
2026-01-26 09:34:00,254.310104,254.459900,253.389999,253.908401,130198
...,...,...,...,...,...
2026-01-30 15:55:00,236.960007,237.070007,236.000000,236.770004,160249
2026-01-30 15:56:00,236.789993,237.000000,236.509995,236.970001,130209
2026-01-30 15:57:00,236.800003,236.919998,236.539993,236.789993,148742


### Phase 1: The Base Signal

The paper first defines VWAP as $$ VWAP_t = \frac{\sum_{i<=t} v_ip_i}{\sum_{i<=t} v_i} $$

and the price difference, or *residual*, defined as $$ d_t = p_t - VWAP_t $$

In order to generalize the price movement within a single minute tick, I will use the following calculation to calculate the `Typical Price` within that minute. $$ \frac{{High} + {Low} + {Close}}{3} $$

In [12]:
# Calculate Typical Price
amd_data['Price'] = (amd_data['High'] + amd_data['Low'] + amd_data['Close']) / 3

# Calculate VWAP (reset everyday)
amd_data['pv'] = amd_data['Price'] * amd_data['Volume']
amd_data['pv_cumsum'] = amd_data.groupby(amd_data.index.date)['pv'].cumsum()
amd_data['vol_cumsum'] = amd_data.groupby(amd_data.index.date)['Volume'].cumsum()
amd_data['VWAP'] = amd_data['pv_cumsum'] / amd_data['vol_cumsum']

# Calculate residual d_t
amd_data['d_t'] = amd_data['Price'] - amd_data['VWAP']
amd_data


,Close,High,Low,Open,Volume,Price,pv,pv_cumsum,vol_cumsum,VWAP,d_t
Datetime,,,,,,,,,,,
2026-01-26 09:30:00,256.459991,258.289886,255.450104,256.744995,1086751,256.733327,2.790052e+08,2.790052e+08,1086751,256.733327,0.000000
2026-01-26 09:31:00,254.625000,256.760010,254.369995,256.320007,178767,255.251668,4.563057e+07,3.246358e+08,1265518,256.524028,-1.272360
2026-01-26 09:32:00,255.300003,255.869995,254.399994,254.625000,156578,255.189997,3.995714e+07,3.645929e+08,1422096,256.377146,-1.187149
2026-01-26 09:33:00,253.750107,255.513397,253.380005,255.490005,160508,254.214503,4.080346e+07,4.053964e+08,1582604,256.157811,-1.943308
2026-01-26 09:34:00,254.310104,254.459900,253.389999,253.908401,130198,254.053335,3.307724e+07,4.384736e+08,1712802,255.997840,-1.944505
...,...,...,...,...,...,...,...,...,...,...,...
2026-01-30 15:55:00,236.960007,237.070007,236.000000,236.770004,160249,236.676671,3.792720e+07,5.748986e+09,23926808,240.273847,-3.597176
2026-01-30 15:56:00,236.789993,237.000000,236.509995,236.970001,130209,236.766663,3.082915e+07,5.779815e+09,24057017,240.254865,-3.488202
2026-01-30 15:57:00,236.800003,236.919998,236.539993,236.789993,148742,236.753332,3.521516e+07,5.815031e+09,24205759,240.233348,-3.480017


### Phase 2: The Filter (Volatility Regime)

The paper states, "some level of volatility is needed for VWAP mean-reversion to work." If it's too flat, there's no opportunity; if it's too volatile, the mean reversion assumption breaks down.

